In [1]:
import sys
import platform
from os.path import join, exists, abspath, dirname
from os import getcwd, makedirs
from glob import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colormaps, cm, colors
import scipy
from scipy.stats import ttest_1samp, pearsonr
from scipy.io import loadmat
from sklearn.metrics import r2_score
import seaborn as sns
import h5py

from tqdm import tqdm

In [2]:
import nibabel as nb
from nilearn import plotting, image
from nipype.interfaces import fsl

In [3]:
dir_current = getcwd().replace('\\','/')

tmp = dir_current.split('/')
idx = [ii for ii, s in enumerate(tmp) if s=='github'][0]

dir_git = '/'.join(tmp[:idx+1])
dir_git

'/home/sungbeenpark/github'

In [4]:
dname = join(dir_git,'nitools')
sys.path.append(dname)
import nitools as nt

In [5]:
dname = join(dir_git,'SUITPy')
sys.path.append(dname)
import SUITPy as suit

In [6]:
dname = join(dir_git)
sys.path.append(dname)
import surfAnalysisPy as surf

In [7]:
dname = join(dir_git,'Functional_Fusion')
sys.path.append(dname)
import Functional_Fusion.atlas_map as am
import Functional_Fusion.reliability as rel

In [8]:
dname = join(dir_git,'PcmPy')
sys.path.append(dname)
import PcmPy as pcm

In [9]:
dname = join(dir_git,'SeqSpatialSupp_fMRI')
sys.path.append(dname)
from SSS import deal_spm
from SSS import util as su
from SSS import plot as splt
from SSS import image as simage
from SSS import glmsingle as ssingle

---

In [10]:
list_run = su.get_list_run()

In [11]:
list_sn = su.get_list_sn()

---

In [12]:
h = 'L'
list_roi = simage.get_list_roi()

---

In [13]:
glm = 1
dir_glm = ssingle.get_dir_glmsingle(glm)
dir_glm

'/mnt/f/SeqSpatialSupp_fMRI/GLMsingle/glm_1'

In [14]:
subj = 'S01'

In [15]:
Xs = ssingle.get_designSINGLE(subj=subj, glm=glm, run=1)
T, P = Xs.shape

---

In [20]:
mask, affine, header = simage.load_mask(subj, glm)
spatial_shape = mask.shape
V = np.prod(spatial_shape)
spatial_shape

(116, 116, 57)

In [ ]:
betas = np.ones((*spatial_shape, P)) * np.nan
for ii, fname in enumerate(tqdm(sorted(glob(join(dir_glm,subj,'beta_*.nii'))))):
    beta = nb.load(fname).get_fdata()
    betas[...,ii] = beta
    
## check the validation
assert betas.shape[-1] == P, f'P mismatch: X has {P} cols, beta has {betas.shape[-1]} regressors'

In [22]:
## Flattened in 2D for vectorized operations
B_2d = betas.reshape(*spatial_shape, P).reshape(V, P)  # (V, P)
del betas

In [23]:
## get y_hat
Y_hat = Xs @ B_2d.T  # (T, V)

yhat_4d = Y_hat.T.reshape(*spatial_shape, T)
output = join(su.get_dir_result(),'%s_Yhat.nii'%subj)
nb.save(nb.Nifti1Image(yhat_4d, affine=affine, header=header), output)

---

In [ ]:
run = 1

In [19]:
y_raw, affine, header = ssingle.get_y_raw(subj=subj, glm=glm, run=run)

## check the validation
assert y_raw.shape[-1] == T, f'T mismatch: X has {T} rows, y has {y_raw.shape[-1]} timepoints'

## Flattened in 2D for vectorized operations
Y_2d = y_raw.reshape(*spatial_shape, T).reshape(V, T).T  # (T, V)
del y_raw

In [ ]:
## get y_res
R = Y_2d - Y_hat  # (T, V)

In [ ]:
voxel = 400000
plt.plot(np.arange(T),Y_2d[:,voxel])
plt.plot(np.arange(T),Y_hat[:,voxel])
# plt.plot(np.arange(T),R[:,voxel])